# Projeto de Telefonia Móvel
### Este projeto tem por objetivo extrair os dados da fatura detalhada da operadora e transformar os dados em informações importantes e insights sobre a conta

##### Importando as bibliotecas necessárias

In [ ]:
!pip install pdfplumber
!pip install pandas
!pip install numpy

In [1]:
import pdfplumber
import re
import pandas as pd
from pathlib import Path
import numpy as np

#### Como a conta detalhada possui muitas páginas, para este projeto foi necessário salvar um arquivo apenas com os dados de cada conta

#### Os dados foram importados do pdf com a biblioteca pdflumber, depois transformados em txt com o auxílio do Path


In [2]:
pdf_url='/home/paula/Downloads/vivotest.pdf'
with pdfplumber.open(pdf_url) as pdf:
    
    first_page = pdf.pages[0]
    pages = pdf.pages
    
    with Path('/home/paula/Downloads/exemplo.txt').open(mode='w') as output_file:
        text=''
        for i,pg in enumerate(pages):
            text+= pages[i].extract_text()
        output_file.write(text)
    

#### Depois de extraídos, os dados precisavam ser padronizados. Com o auxílio da biblioteca RE, foi criado uma expressão regular para localizar os textos que continham formatos diferentes, logo depois foram removidas informações irrelevantes do plano para este estudo através do método replace.

In [3]:
busca='SMART EMPRESAS'
dadosFatura=''
dadosGerais=''
texto=''
with open('/home/paula/Downloads/exemplo.txt', 'r', encoding = 'utf-8') as arquivo:
        
    for linha in arquivo: 
        strug=' TE'
        strug1='MAS'
        if busca in linha:
                if (re.search(r'\s[0-9]*\s',linha)):
                    if strug in linha:
                        texto= linha.replace(' TE', '')
                    elif strug1 in linha:
                        texto= linha.replace(' MAS', '')
                    else:
                        texto= linha
                    dadosGerais+=texto
                elif strug in linha:
                    texto= linha.replace(' TE', '')
                    dadosFatura+=texto
                
                elif strug1 in linha:
                    texto= linha.replace(' MAS', '')
                    dadosFatura+=texto
                    
                else:
                    dadosFatura+=linha
                
                
                with Path('/home/paula/Downloads/dadosGeraisPlanos.txt').open(mode='w') as output_file:
                    output_file.write(dadosGerais)
               
                with Path('/home/paula/Downloads/dadosFatura.txt').open(mode='w') as output_file:
                    output_file.write(dadosFatura)

#### Os dados depois de tratados foram armazenados em arquivos diferentes para fazer a composição de dois dataframes bases relacionados à fatura.

In [6]:
dfFatura = pd.read_csv('/home/paula/Downloads/dadosFatura.txt', sep="\r\n", header=None, names=["coluna"]) #names=["number", "tipo", "c"] 


/tmp/ipykernel_4308/3877272684.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dfFatura = pd.read_csv('/home/paula/Downloads/dadosFatura.txt', sep="\r\n", header=None, names=["coluna"]) #names=["number", "tipo", "c"]


In [7]:
divisao=dfFatura["coluna"].str.split(" ")

In [8]:
numero=divisao.str.get(0)
desc1=divisao.str.get(1)
desc2=divisao.str.get(2)
plan=divisao.str.get(3)
period1=divisao.str.get(4)
period2=divisao.str.get(5)
period3=divisao.str.get(6)
price=divisao.str.get(7)

In [9]:
dfFatura['Numero']=numero
dfFatura['desc1']=desc1
dfFatura['desc2']=desc2
dfFatura['internet']=plan
dfFatura['Descrição'] = dfFatura['desc1'] + " " + dfFatura['desc2'] + " " + dfFatura['internet']
dfFatura['Inicio']=period1
dfFatura['a']=period2
dfFatura['Final']=period3
dfFatura['Valor']=price.str.replace(',', '.') #convertendo pontuação
dfFatura['Valor']=dfFatura['Valor'].astype('float64')

In [10]:
dfFatura.drop(['coluna'],axis=1,inplace=True)
dfFatura.drop(['a'],axis=1,inplace=True)
dfFatura.drop(['desc1'],axis=1,inplace=True)
dfFatura.drop(['desc2'],axis=1,inplace=True)
dfFatura.drop(['internet'],axis=1,inplace=True)
dfFatura.head()

,Numero,Descrição,Inicio,Final,Valor
0,24-98124-6525,SMART EMPRESAS 0.2GB,06/09/22,05/10/22,35.3
1,24-98125-2580,SMART EMPRESAS 0.2GB,06/09/22,05/10/22,35.3
2,24-98140-6666,SMART EMPRESAS 0.2GB,06/09/22,05/10/22,35.3
3,24-98140-6667,SMART EMPRESAS 0.2GB,06/09/22,05/10/22,35.3
4,24-98140-6668,SMART EMPRESAS 0.2GB,06/09/22,05/10/22,35.3


In [12]:
dfPlan = pd.read_csv('/home/paula/Downloads/dadosGeraisPlanos.txt', sep="\r\n", header=None, names=["dados"]) #names=["number", "tipo", "c"]
divisao=dfPlan["dados"].str.split(" ") 

/tmp/ipykernel_4308/3799240282.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dfPlan = pd.read_csv('/home/paula/Downloads/dadosGeraisPlanos.txt', sep="\r\n", header=None, names=["dados"]) #names=["number", "tipo", "c"]


In [13]:
desc=divisao.str.get(0)
desc0=divisao.str.get(1)
internet=divisao.str.get(2)
quantidade=divisao.str.get(3)

In [15]:
dfPlan['desc1']=desc
dfPlan['desc2']=desc0
dfPlan['Descrição'] = dfPlan['desc1'] + " " + dfPlan['desc2']
dfPlan['Internet']=internet
dfPlan['Quantidade']=quantidade
dfPlan['Quantidade']=dfPlan['Quantidade'].astype('int32')
dfPlan.head()

,dados,desc1,desc2,Descrição,Internet,Quantidade
0,SMART EMPRESAS 3GB 181,SMART,EMPRESAS,SMART EMPRESAS,3GB,181
1,SMART EMPRESAS 1GB 105,SMART,EMPRESAS,SMART EMPRESAS,1GB,105
2,SMART EMPRESAS 1GB 50,SMART,EMPRESAS,SMART EMPRESAS,1GB,50
3,SMART EMPRESAS 25GB 32,SMART,EMPRESAS,SMART EMPRESAS,25GB,32
4,SMART EMPRESAS 2GB 25,SMART,EMPRESAS,SMART EMPRESAS,2GB,25


In [16]:
dfPlan.drop(['dados'],axis=1,inplace=True)
dfPlan.drop(['desc1'],axis=1,inplace=True)
dfPlan.drop(['desc2'],axis=1,inplace=True)
dfPlan.head()

,Descrição,Internet,Quantidade
0,SMART EMPRESAS,3GB,181
1,SMART EMPRESAS,1GB,105
2,SMART EMPRESAS,1GB,50
3,SMART EMPRESAS,25GB,32
4,SMART EMPRESAS,2GB,25
